In [ ]:
%load_ext lab_black

In [ ]:
experiment_id = 0
use_gpr = False

In [ ]:
"""Setup the environment."""
from pathlib import Path
import os
import sys

import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import ROIS, EXPERIMENTS
from brainweb_tdcs.study import get_experiments_for_roi, log_marginal_likelihood
from brainweb_tdcs.plot import (
    display_side_by_side,
    plot_conductivity_categorical,
    plot_posterior,
)

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

# Set the random seed
RANDOM_SEED = 1234

In [ ]:
"""Set some constants."""
CAPTIONS = {
    "e": "Magnitude of the\nelectric field",
    "e_r": "Magnitude of the\nnormal component of electric field",
}
LABELS = {
    "e": "$| \mathbf{e} |$ (mVm$^{-1}$)",
    "e_r": "$| \mathbf{e}_r |$ (mVm$^{-1}$)",
}
FUNC_DICT = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

In [ ]:
"""Select the experiment."""
experiment = EXPERIMENTS[experiment_id]
display(
    Markdown(
        "# Evaluate the effect of the conductivity profile "
        f"({experiment.roi}, {experiment.montage} electrodes montage)"
    )
)

In [ ]:
display(
    HTML(
        f"""
<div style="background: #efefef; color: #5f5f5f; padding: 15pt;">
    <b style="display: inline-block; width: 120pt;">Region of interest</b> {experiment.roi.long_name} ({experiment.roi})</br>
    <b style="display: inline-block; width: 120pt;">Electrodes montage</b> {experiment.montage} ({"Bipolar" if experiment.is_bipolar else "Unipolar"})</br>
    </br>
    In this notebook, we evaluate the effect of the conductivity profile on the electric field computed in the region of interest resulting from the stimulation using the {'uniform distribution' if use_gpr else 'realistic distribution'}.
</div>
"""
    )
)

## Data

First, we load the data corresponding to this experiment and display the measurements of the average absolute mangitude of the electric field $\bar{| \pmb{e} |}$ and of its component normal to the cortical surface $\bar{| \pmb{e}_r |}$.

In [ ]:
"""Load the experiment results."""
if use_gpr:
    data = experiment.get_gpr_data()[["sub", "k", "k_id", "e", "e_r"]]
else:
    data = experiment.get_data()[["sub", "k", "k_id", "e", "e_r"]]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for i, voi in enumerate(("e", "e_r")):
    plot_conductivity_categorical(
        axs[i], voi, experiment, data, LABELS[voi], CAPTIONS[voi]
    )
plt.show()
columns = ["mean", "std", "min", "25%", "75%", "max"]
display_side_by_side(
    [
        data.groupby("k").describe()["e"][columns],
        data.groupby("k").describe()["e_r"][columns],
    ],
    ["", ""],
)

The data regarding the conductivity profile is already formatted. The `"k"` column contains the name of the conductivity profile (`"reference"` and `"halton_i"`) while the column `"k_id"` contains an integer ranging from $0$ to $20$ included ($0$ corresponding to the reference conductivity profile).

## Models

To compare the measurements acquired with the different conductivity profiles, we use a Bayesian linear model:
$$
Y \sim \mathcal{N}(\mu, \sigma^2),
$$
where $Y$ is the dependent variable we focus on and
$$
\mu = \alpha + \sum_{k=1}^{20} \beta_k \cdot X_k + \varepsilon
$$
with $\alpha$ the intercept, $\pmb{\beta} = [\beta_1, \; \dots, \: \beta_{20}]$ the slopes, $\pmb{X} = [X_1, \: \dots, \: X_{20}]^\top$ the vector of independent variables and $\varepsilon$ the error term. The values of $X_k$ are either $1$ if the value in `"k_id"` is equal to $k$ or $0$ otherwise. This way, the reference population is described by $\alpha + \varepsilon$ and the values of each $\beta_k$ correspond to the difference between the reference population and the measurements resulting from the $k$-th conductivity profile.

This model is referred to as the "*pooled model*" because it does not account for the hierarchy of the data (*i.e* some measurements belong correspond to the same subject). Hence, we also consider another model that we refer to as the "*hierarchic model*" which is defined as
$$
Y \sim \mathcal{N}(\mu_i, \sigma^2), \forall i \in [1, 20]
$$
where $i$ corresponds to the $i$-th subject and
$$
\mu_i = \alpha_i + \sum_{k=1}^{20} (\beta_k)_i \cdot X_k + \varepsilon.
$$
The definition of both the intercept and the slopes thus depend on the subject and become
$$
\begin{split}
    \alpha_i &= \alpha^{(com)} + \alpha_i^{(sub)}, \\
    (\beta_k)_i &= \beta_k^{(com)} + (\beta_k^{(sub)})_i,
\end{split}
$$
with $\alpha^{(com)}$ and $\beta_k^{(com)}$ respectively the common part of the intercept and of the slopes and $\alpha_i^{(sub)}$ and $(\beta_k^{(sub)})_i$ the subject specific part of the intercept and of the slopes.

For both the pooled and the hierarchic models, weakly informative priors are set (Westfall, 2017). They are then fitted using the NUTS sampler (Hoffman *et al.*, 2011).

In [ ]:
"""Define model storages."""
models = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
fits = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
summaries = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
rope_vs_hdi = dict(
    e=dict(pooled=dict(), hierarchic=dict()), e_r=dict(pooled=dict(), hierarchic=dict())
)

## Pooled model

In [ ]:
"""Define models."""
for voi in ("e", "e_r"):
    models[voi]["pooled"] = bmb.Model(f"{voi} ~ C(k_id)", data)

In [ ]:
"""Fit models."""
for voi in ("e", "e_r"):
    fits[voi]["pooled"] = models[voi]["pooled"].fit(
        draws=1000, chains=4, random_seed=RANDOM_SEED
    )

In [ ]:
"""Summarize results."""
indices = ["α", *[f"β_{k}" for k in data["k"].unique()[1:]], "σ"]
for voi in ("e", "e_r"):
    summaries[voi]["pooled"] = az.summary(
        fits[voi]["pooled"], stat_funcs=FUNC_DICT, extend=False
    )
    summaries[voi]["pooled"].index = indices

In [ ]:
"""Plot betas for the pooled models."""
n_beta = len(indices[1:-1])
fig, axs = plt.subplots(n_beta, 2, figsize=(10, n_beta * 1.5), sharex="col")
for i, beta in enumerate(indices[1:-1]):
    suffix = beta.split("_")[1]
    for j, voi in enumerate(("e", "e_r")):
        rope_vs_hdi[voi]["pooled"][beta] = plot_posterior(
            axs[i][j],
            experiment,
            fits[voi]["pooled"],
            summaries[voi]["pooled"],
            "C(k_id)",
            LABELS[voi] if i == n_beta - 1 else "",
            CAPTIONS[voi] if i == 0 else "",
            i,
            summary_param=beta,
            show_zero=True,
            beta_suffix=suffix,
            rope_width=0.1 * data[voi].std(),
        )
fig.tight_layout()
plt.show()
display_side_by_side(
    [
        pd.concat(
            (
                summaries[voi]["pooled"],
                pd.DataFrame.from_dict(
                    rope_vs_hdi[voi]["pooled"],
                    orient="index",
                    columns=["HDI ⊂ ROPE", "ROPE ⊂ HDI"],
                ),
            ),
            axis=1,
        )
        for voi in ("e", "e_r")
    ],
    ["", ""],
)

## Hierarchic model

In [ ]:
"""Define the models."""
for voi in ("e", "e_r"):
    models[voi]["hierarchic"] = bmb.Model(f"{voi} ~ C(k_id) + (C(k_id) | sub)", data)

In [ ]:
"""Fit the models."""
for voi in ("e", "e_r"):
    fits[voi]["hierarchic"] = models[voi]["hierarchic"].fit(
        draws=1000, chains=4, random_seed=RANDOM_SEED, target_accept=0.9
    )

In [ ]:
"""Summarize the results."""
indices = [
    "α^(com)",
    *[f"β^(com)_{k}" for k in data["k"].unique()[1:]],
    "σ(α^(sub))",
    *[f"α^(sub)_{i}" for i in range(len(data["sub"].unique()))],
    *[f"σ(β^(sub)_{k})" for k in data["k"].unique()[1:]],
    *[
        f"(β^(sub)_{k})_{i}"
        for k in data["k"].unique()[1:]
        for i in range(len(data["sub"].unique()))
    ],
    "σ",
]
for voi in ("e", "e_r"):
    summaries[voi]["hierarchic"] = az.summary(
        fits[voi]["hierarchic"], stat_funcs=FUNC_DICT, extend=False
    )
    summaries[voi]["hierarchic"].index = indices

In [ ]:
"""Plot betas for the hierarchic models."""
n_beta = len(indices[1:21])
fig, axs = plt.subplots(n_beta, 2, figsize=(10, n_beta * 1.5), sharex="col")
for i, beta in enumerate(indices[1:21]):
    suffix = beta.split("_")[1]
    for j, voi in enumerate(("e", "e_r")):
        rope_vs_hdi[voi]["hierarchic"][beta] = plot_posterior(
            axs[i][j],
            experiment,
            fits[voi]["hierarchic"],
            summaries[voi]["hierarchic"],
            "C(k_id)",
            LABELS[voi] if i == n_beta - 1 else "",
            CAPTIONS[voi] if i == 0 else "",
            i,
            summary_param=beta,
            show_zero=True,
            beta_suffix=suffix,
            rope_width=0.1 * data[voi].std(),
        )
fig.tight_layout()
plt.show()
rows = ["α^(com)", "σ(α^(sub))"]
for i in range(1, 21):
    rows += [f"β^(com)_halton_{i}", f"σ(β^(sub)_halton_{i})"] if not use_gpr else [f"β^(com)_gpr_{i}", f"σ(β^(sub)_gpr_{i})"]
rows += "σ"
display_side_by_side(
    [
        pd.concat(
            (
                summaries[voi]["hierarchic"].loc[rows, :],
                pd.DataFrame.from_dict(
                    rope_vs_hdi[voi]["hierarchic"],
                    orient="index",
                    columns=["HDI ⊂ ROPE", "ROPE ⊂ HDI"],
                ),
            ),
            axis=1,
        )
        for voi in ("e", "e_r")
    ],
    ["", ""],
)

## References

- Hoffman *et al.* (2011) - https://arxiv.org/abs/1111.4246
- Westfall (2017) - https://arxiv.org/abs/1702.01201